In [1]:
import torch
import torch._dynamo
from torch import nn

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32, 64)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.nn.functional.gelu(x)
        return x

model = MLP()

batch_size = 8
x = torch.randn(batch_size, 32)

In [ ]:
from torch.fx import passes, symbolic_trace
model = symbolic_trace(fn)

g = passes.graph_drawer.FxGraphDrawer(model, 'fn')
with open("unoptimized_graph.svg", "wb") as f:
    f.write(g.get_dot_graph().create_svg())

In [21]:
import torch._dynamo
from torch._functorch.aot_autograd import aot_module_simplified
from functorch.compile import compiled_function, draw_graph


def toy_backend(gm, sample_inputs): 
    def fw(gm, sample_inputs):
        draw_graph(gm, "forward.svg")
        return gm.forward
    
    def bw(gm, sample_inputs):
        draw_graph(gm, "backward.svg")
        return gm.forward

    # Invoke AOTAutograd
    return aot_module_simplified(
        gm,
        sample_inputs,
        fw_compiler=fw,
        bw_compiler=bw
    )

def fn(x):
    return x**2

model = fn
x = torch.tensor(5., requires_grad=True)

torch._dynamo.reset()
cmodel = torch.compile(model, backend=toy_backend, dynamic=True)

# triggers compilation of forward graph on the first run
out = cmodel(x).sum().backward()

Writing FX graph to file: forward.svg
Writing FX graph to file: backward.svg


/opt/conda/lib/python3.10/site-packages/torch/_functorch/aot_autograd.py:1251: UserWarning: Your compiler for AOTAutograd is returning a a function that doesn't take boxed arguments. Please wrap it with functorch.compile.make_boxed_func or handle the boxed arguments yourself. See https://github.com/pytorch/pytorch/pull/83137#issuecomment-1211320670 for rationale.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/_functorch/aot_autograd.py:1251: UserWarning: Your compiler for AOTAutograd is returning a a function that doesn't take boxed arguments. Please wrap it with functorch.compile.make_boxed_func or handle the boxed arguments yourself. See https://github.com/pytorch/pytorch/pull/83137#issuecomment-1211320670 for rationale.
  warnings.warn(


In [5]:
print(symtraced.code)




def forward(self, x):
    param = self.param
    add = x + param;  x = param = None
    linear = self.linear(add);  add = None
    clamp = linear.clamp(min = 0.0, max = 1.0);  linear = None
    return clamp
    
